In [6]:
import librosa
import numpy as np
import os
import math
import pickle
from sklearn.cluster import KMeans
import hmmlearn.hmm

In [7]:
def get_mfcc(file_path):
  y, sr = librosa.load(file_path) # read .wav file
  hop_length = math.floor(sr*0.010) # 10ms hop
  win_length = math.floor(sr*0.025) # 25ms frame
  # mfcc is 12 x T matrix
  mfcc = librosa.feature.mfcc(
      y, sr, n_mfcc=12, n_fft=1024,
      hop_length=hop_length, win_length=win_length)
  # substract mean from mfcc --> normalize mfcc
  mfcc = mfcc - np.mean(mfcc, axis=1).reshape((-1,1)) 
  # delta feature 1st order and 2nd order
  delta1 = librosa.feature.delta(mfcc, order=1)
  delta2 = librosa.feature.delta(mfcc, order=2)
  # X is 36 x T
  X = np.concatenate([mfcc, delta1, delta2], axis=0) # O^r
  # return T x 36 (transpose of X)
  return X.T # hmmlearn use T x N matrix

In [8]:
def get_class_data(data_dir):
  files = os.listdir(data_dir)
  mfcc = [get_mfcc(os.path.join(data_dir,f)) for f in files if f.endswith(".wav")]
  return mfcc

In [9]:
def clustering(X, n_clusters=20):
  kmeans = KMeans(n_clusters=n_clusters, n_init=50, random_state=0, verbose=0)
  kmeans.fit(X)
  print("centers", kmeans.cluster_centers_.shape)
  return kmeans  

In [13]:
class_names = ["toi", "nguoi", "dich",  "theo", "benh_nhan"]#, "test_toi", "test_nguoi", "test_theo", "test_dich", "test_benh_nhan"]
dataset = {}
train_dataset = {}
for cname in class_names:
  dataset[cname] = get_class_data(os.path.join("data", cname))
  if cname[:4] != "test":
#         print(f"Load {cname} dataset to train")
    train_dataset[cname] = get_class_data(os.path.join("data", cname))

#   # Get all vectors in the datasets
#   all_vectors = np.concatenate([np.concatenate(v, axis=0) for k, v in dataset.items()], axis=0)
#   print("vectors", all_vectors.shape)
#   # Run K-Means algorithm to get clusters
#   kmeans = clustering(all_vectors)
#   print("centers", kmeans.cluster_centers_.shape)

  # Get all vectors in the datasets
all_train_vectors = np.concatenate([np.concatenate(v, axis=0) for k, v in train_dataset.items()], axis=0)
print("vectors", all_train_vectors.shape)
  # Run K-Means algorithm to get clusters
kmeans = clustering(all_train_vectors)
print("centers", kmeans.cluster_centers_.shape)

models = {}
for cname in class_names:
  print("====", cname)
  class_vectors = dataset[cname]
    # convert all vectors to the cluster index
    # dataset['cname'] = [O^1, ... O^R]
    # O^r = (c1, c2, ... ct, ... cT)
    # O^r size T x 1
  dataset[cname] = list([kmeans.predict(v).reshape(-1,1) for v in dataset[cname]])
    
    
  if cname == "toi":
    # =================================================================
    # toi |t|~|o|~|i|
    hmm = hmmlearn.hmm.MultinomialHMM(
      n_components=9, init_params='e', params='ste', verbose=True
    ) 
    hmm.startprob_ = np.array([0.6, 0.2, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
    hmm.transmat_ = np.array([
        [0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4, 0.6],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
      ])

  elif cname == "nguoi":
    # =================================================================
    # nguoi |ng|~|uo|~|i|
    hmm = hmmlearn.hmm.MultinomialHMM(
      n_components=15, init_params='e', params='ste', verbose=True
    )
    hmm.startprob_ = np.array([0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
    hmm.transmat_ = np.array([
        [0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0],    
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0],    
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2],    
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
      ])

  elif cname == "dich":
    # =================================================================
    # dich |d|~|i|~|ch|
    hmm = hmmlearn.hmm.MultinomialHMM(
      n_components=9, init_params='e', params='ste', verbose=True
    )
    hmm.startprob_ = np.array([0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
    hmm.transmat_ = np.array([
        [0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
      ])

  elif cname == "theo":
    # =================================================================
    # theo |th|~|e|~|o|
    hmm = hmmlearn.hmm.MultinomialHMM(
      n_components=9, init_params='e', params='ste', verbose=True
    )
    hmm.startprob_ = np.array([0.6, 0.2, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
    hmm.transmat_ = np.array([
        [0.1, 0.3, 0.6, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.2, 0.3, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.3, 0.3, 0.4, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.3, 0.3, 0.4, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.4, 0.5, 0.1, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
      ])

  elif cname == "benh_nhan":
    # =================================================================
    # benh_nhan |b|~|e|~|nh|~|silent|~|nh|~|a|~|n|  
    hmm = hmmlearn.hmm.MultinomialHMM(
        n_components=21, init_params='e', params='ste', verbose=True
    )
    hmm.startprob_ = np.array([0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
    hmm.transmat_ = np.array([
        [0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],    
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],    
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],    
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
    ])

  if cname[:4] != "test":
    X = np.concatenate(dataset[cname])
    lengths = list([len(x) for x in dataset[cname]])
#       print("training class", cname)
#       print(X.shape, lengths, len(lengths))
    hmm.fit(X, lengths=lengths)
#       print(hmm.transmat_)
    models[cname] = hmm
    with open(os.path.join("models", cname + ".pkl"), "wb") as file: 
      pickle.dump(models[cname], file)
      print("Training done. Model has been dump to ", os.path.join("models", cname + ".pkl"))


#TEST
#   print("Testing and Labeling")
#   for true_cname in class_names:
#     if true_cname[:4] == "test":
#       print("==================================")
#       print(true_cname)
#       print("==================================")
    
#       lname = true_cname[5:]
#       totalWord = 0
#       true = 0
#       accuracy = 0
    
#       for O in dataset[true_cname]:
#         totalWord += 1
#         scores = {}
#         for cname, model in models.items():
#           if cname[:4] != "test":
#             score = model.score(O, [len(O)])
#             scores[cname] = score
#         print(scores)
#         srt = sorted(scores.items(), key=lambda x: x[1], reverse=True)
#         print(srt[0])




#         if srt[0][0] == lname:
#             true += 1
#       accuracy = true/totalWord
#       print("--------------------------------------------")
#       print("!note: test_folder must contain wavs that it records exactly the word which be trained")
#       print("accuracy: ", accuracy, ",true: ", true, ",total_word: ", totalWord)

/home/apple/anaconda3/envs/voice/lib/python3.7/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


vectors (29978, 36)
centers (20, 36)
centers (20, 36)
==== toi


         1      -12180.5409             +nan
         2       -9229.2084       +2951.3324
         3       -7586.4364       +1642.7721
         4       -7225.0991        +361.3373
         5       -7167.1107         +57.9884
         6       -7130.7158         +36.3949
         7       -7078.3109         +52.4048
         8       -7026.9081         +51.4028
         9       -6988.6667         +38.2414
        10       -6961.1210         +27.5457


Training done. Model has been dump to  <_io.BufferedWriter name='models/toi.pkl'>
==== nguoi


         1      -17085.8758             +nan
         2      -10356.0565       +6729.8193
         3       -8306.3339       +2049.7227
         4       -7804.7835        +501.5504
         5       -7573.2256        +231.5579
         6       -7447.8376        +125.3880
         7       -7377.5747         +70.2629
         8       -7337.2684         +40.3063
         9       -7310.3510         +26.9174
        10       -7292.8236         +17.5273


...

         2      -11205.2633       +3461.1319
         3      -10200.6207       +1004.6425
         4       -9708.1175        +492.5033
         5       -9219.5154        +488.6021
         6       -8952.7788        +266.7366
         7       -8769.8414        +182.9374
         8       -8605.4377        +164.4037
         9       -8543.8820         +61.5558
        10       -8513.3563         +30.5257


Training done. Model has been dump to  <_io.BufferedWriter name='models/theo.pkl'>
==== benh_nhan


         1      -29274.3817             +nan
         2      -19165.4992      +10108.8825
         3      -16284.4320       +2881.0673
         4      -15361.1743        +923.2577
         5      -14902.9714        +458.2029
         6      -14660.9362        +242.0353
         7      -14530.2681        +130.6681
         8      -14449.9222         +80.3459
         9      -14402.2752         +47.6470


Training done. Model has been dump to  <_io.BufferedWriter name='models/benh_nhan.pkl'>


        10      -14370.8529         +31.4223


In [38]:
class_names = ["toi", "nguoi", "dich",  "theo", "benh_nhan","test_random"]#,"test_toi", "test_nguoi", "test_theo", "test_dich", "test_benh_nhan"]
dataset = {}
train_dataset = {}
for cname in class_names:
    dataset[cname] = get_class_data(os.path.join("data", cname))
    if cname[:4] != "test":
#         print(f"Load {cname} dataset to train")
        train_dataset[cname] = get_class_data(os.path.join("data", cname))

#   # Get all vectors in the datasets
#   all_vectors = np.concatenate([np.concatenate(v, axis=0) for k, v in dataset.items()], axis=0)
#   print("vectors", all_vectors.shape)
#   # Run K-Means algorithm to get clusters=============================
    # theo |th|~|e|~|o|
#   kmeans = clustering(all_vectors)
#   print("centers", kmeans.cluster_centers_.shape)

# Get all vectors in the datasets
all_train_vectors = np.concatenate([np.concatenate(v, axis=0) for k, v in train_dataset.items()], axis=0)
print("vectors", all_train_vectors.shape)
# Run K-Means algorithm to get clusters
kmeans = clustering(all_train_vectors)
print("centers", kmeans.cluster_centers_.shape)

models = {}
for cname in class_names:
    class_vectors = dataset[cname]
# convert all vectors to the cluster index
# dataset['cname'] = [O^1, ... O^R]
# O^r = (c1, c2, ... ct, ... cT)
# O^r size T x 1
    dataset[cname] = list([kmeans.predict(v).reshape(-1,1) for v in dataset[cname]])


    if cname == "toi":
# =================================================================
# toi |t|~|o|~|i|
        hmm = hmmlearn.hmm.MultinomialHMM(
          n_components=9, init_params='e', params='ste', verbose=True
        ) 
        hmm.startprob_ = np.array([0.6, 0.2, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
        hmm.transmat_ = np.array([
            [0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4, 0.6],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
          ])

    elif cname == "nguoi":
# =================================================================
# nguoi |ng|~|uo|~|i|
        hmm = hmmlearn.hmm.MultinomialHMM(
          n_components=15, init_params='e', params='ste', verbose=True
        )
        hmm.startprob_ = np.array([0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
        hmm.transmat_ = np.array([
            [0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0],    
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0],    
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2],    
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
          ])

    elif cname == "dich":
    # =================================================================
    # dich |d|~|i|~|ch|
        hmm = hmmlearn.hmm.MultinomialHMM(
          n_components=9, init_params='e', params='ste', verbose=True
        )
        hmm.startprob_ = np.array([0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
        hmm.transmat_ = np.array([
            [0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
          ])

    elif cname == "theo":
    # =================================================================
    # theo |th|~|e|~|o|
        hmm = hmmlearn.hmm.MultinomialHMM(
          n_components=9, init_params='e', params='ste', verbose=True
        )
        hmm.startprob_ = np.array([0.6, 0.2, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
        hmm.transmat_ = np.array([
            [0.1, 0.3, 0.6, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.2, 0.3, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.3, 0.3, 0.4, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.3, 0.3, 0.4, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.4, 0.5, 0.1, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
          ])

    elif cname == "benh_nhan":
    # =================================================================
    # benh_nhan |b|~|e|~|nh|~|silent|~|nh|~|a|~|n|  
        hmm = hmmlearn.hmm.MultinomialHMM(
            n_components=21, init_params='e', params='ste', verbose=True
        )
        hmm.startprob_ = np.array([0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
        hmm.transmat_ = np.array([
            [0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],    
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],    
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],    
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
        ])

    if cname[:4] != "test":
      X = np.concatenate(dataset[cname])
      lengths = list([len(x) for x in dataset[cname]])
    #       print("training class", cname)
    #       print(X.shape, lengths, len(lengths))
      hmm.fit(X, lengths=lengths)
    #       print(hmm.transmat_)
      models[cname] = hmm
    #       print("Training done")



vectors (16371, 36)
centers (20, 36)
centers (20, 36)


         1       -8754.1479             +nan
         2       -6446.0342       +2308.1137
         3       -5578.2484        +867.7857
         4       -4990.5577        +587.6907
         5       -4769.7415        +220.8162
         6       -4720.8443         +48.8972
         7       -4700.0539         +20.7904
         8       -4687.7372         +12.3167
         9       -4675.5109         +12.2264
        10       -4663.1413         +12.3696
         1       -9992.3091             +nan
         2       -6386.7892       +3605.5199
         3       -5253.4023       +1133.3870
         4       -4879.4107        +373.9915
         5       -4620.1410        +259.2698
         6       -4434.4411        +185.6998
         7       -4348.6086         +85.8326
         8       -4280.2054         +68.4032
         9       -4225.6521         +54.5533
        10       -4188.0596         +37.5926
         1       -7718.3879             +nan
         2       -6138.4674       +1579.9204
         3

In [20]:
for name in class_names:
    print(name)
#     np.set_printoptions(precision=3, supress=True)
    print(models[name].transmat_)

toi
[[0.90554024 0.0282497  0.06621006 0.         0.         0.
  0.         0.         0.        ]
 [0.         0.73654782 0.07773365 0.18571853 0.         0.
  0.         0.         0.        ]
 [0.         0.         0.84957501 0.04489667 0.10552832 0.
  0.         0.         0.        ]
 [0.         0.         0.         0.6342558  0.1837118  0.1820324
  0.         0.         0.        ]
 [0.         0.         0.         0.         0.87480763 0.00779502
  0.11739735 0.         0.        ]
 [0.         0.         0.         0.         0.         0.86065566
  0.07902221 0.06032214 0.        ]
 [0.         0.         0.         0.         0.         0.
  0.90797495 0.08943263 0.00259242]
 [0.         0.         0.         0.         0.         0.
  0.         0.90616774 0.09383226]
 [0.         0.         0.         0.         0.         0.
  0.         0.         1.        ]]
nguoi
[[9.34828032e-01 1.54092282e-02 4.97627402e-02 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.000000

In [16]:

  print("Testing and Labeling")
  for true_cname in class_names:
    if true_cname[:4] == "test":
      print("==================================")
      print(true_cname)
      print("==================================")
    
      lname = true_cname[5:]
      totalWord = 0
      true = 0
      accuracy = 0
    
      for O in dataset[true_cname]:
        totalWord += 1
        scores = {}
        for cname, model in models.items():
          if cname[:4] != "test":
            score = model.score(O, [len(O)])
            scores[cname] = score
        print(scores)
        srt = sorted(scores.items(), key=lambda x: x[1], reverse=True)
        print(srt[0])

KeyError: 'toi'

In [13]:
from joblib import *

In [14]:
def save_mod(cname, model):
    dump(model, f"{cname}.pkl")
    print(f"Save model {cname} success")

In [15]:
save_mod('theo', models['theo'])

Save model theo success


In [39]:
print("Testing and Labeling")
for true_cname in class_names:
    if true_cname[:4] == "test":
      print("==================================")
      print(true_cname)
      print("==================================")

      lname = true_cname[5:]
      totalWord = 0
      true = 0
      accuracy = 0

      for O in dataset[true_cname]:
        totalWord += 1
        scores = {}
        for cname, model in models.items():
          if cname[:4] != "test":
            score = model.score(O, [len(O)])
            scores[cname] = score
        print(scores)
        srt = sorted(scores.items(), key=lambda x: x[1], reverse=True)
        print(srt[0])

Testing and Labeling
test_random
{'toi': -371.7180337122106, 'nguoi': -433.22707464687085, 'dich': -368.46615442297735, 'theo': -483.83681189046877, 'benh_nhan': -223.30007940437494}
('benh_nhan', -223.30007940437494)
